In [92]:
import sys

In [93]:
sys.version

'3.7.0 (default, Jun 28 2018, 07:39:16) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

In [95]:
#!conda install python==3.7 --y

#!pip3 uninstall spacy --y

#!conda install -c conda-forge spacy --y

#!conda install -c anaconda gensim --y

In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

In [2]:
df_1 = pd.read_csv("/Users/sreeharshaankem/MachineLearning/pytorch_nlp_book/chapter_3/data/imdb_reviews_train.csv",
                  index_col = "Unnamed: 0")
df_2 = pd.read_csv("/Users/sreeharshaankem/MachineLearning/pytorch_nlp_book/chapter_3/data/imdb_reviews_test.csv",
                  index_col = "Unnamed: 0")

In [3]:
df = pd.concat([df_1, df_2])
df.head()

,review,sentiment
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


In [6]:
import re
import string
import spacy

nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser', 'tagger'])
all_stopwords = list(nlp.Defaults.stop_words)
stop_punctuations = list(string.punctuation)
all_stopwords.extend(stop_punctuations)
all_stopwords.extend(["/><br", "---", "...", "-pron-"])



In [17]:
# Preprocessing
def tokenizer(document):
    
    # Convert to lower case
    tokens = [token.lower() for token in document.split()] 
    
    # Remove characters other than alphabets
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    
    # Remove stop words and punctuations.
    tokens = [token for token in tokens if token not in all_stopwords]
    
    # Remove empty strings, if any. 
    tokens = [token for token in tokens if token != ""]
    return tokens


In [19]:
from tqdm import tqdm

tqdm.pandas(desc="my bar!")

df["review_processed"] = df["review"].progress_apply(tokenizer)

/Users/sreeharshaankem/.local/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
my bar!: 100%|██████████| 50000/50000 [00:51<00:00, 971.27it/s] 


In [20]:
df["review_processed"].values.tolist()

,review,sentiment,review_processed
0,For a movie that gets no respect there sure ar...,1,"[movie, gets, respect, sure, lot, memorable, q..."
1,Bizarre horror movie filled with famous faces ...,1,"[bizarre, horror, movie, filled, famous, faces..."
2,"A solid, if unremarkable film. Matthau, as Ein...",1,"[solid, unremarkable, film, matthau, einstein,..."
3,It's a strange feeling to sit alone in a theat...,1,"[strange, feeling, sit, theater, occupied, par..."
4,"You probably all already know this by now, but...",1,"[probably, know, additional, episodes, aired, ..."


# Train Word2Vec Model

In [127]:
model = Word2Vec(
          size = 100,
          window = 5,
          min_count = 50,
          max_vocab_size = 5000,
          seed = 9,
          workers = 3
          )
#model.save("word2vec.model")

In [128]:
model.build_vocab(sentences = df["review_processed"].values.tolist())

In [129]:
model.train(sentences = df["review_processed"].values.tolist(), 
            total_examples=50000, 
            epochs=1000,
            )

(2405002410, 5336996000)

# Checking for similar words to a given word

In [130]:
model.wv.most_similar("good")

[('decent', 0.7609848380088806),
 ('bad', 0.6843814849853516),
 ('pretty', 0.6568313837051392),
 ('ok', 0.6312131285667419),
 ('great', 0.6203598380088806),
 ('nice', 0.5505648255348206),
 ('better', 0.5255361795425415),
 ('acting', 0.5113769769668579),
 ('fine', 0.5030808448791504),
 ('cool', 0.49645939469337463)]

In [131]:
model.wv.most_similar("actor")

[('role', 0.8072347640991211),
 ('actress', 0.7410792708396912),
 ('roles', 0.7387673854827881),
 ('performance', 0.697562575340271),
 ('actors', 0.6307398080825806),
 ('plays', 0.5915753245353699),
 ('playing', 0.5831061601638794),
 ('john', 0.5820398926734924),
 ('casting', 0.5749509334564209),
 ('robert', 0.5682957768440247)]

In [132]:
model.wv.most_similar("camera")

[('shots', 0.6942334175109863),
 ('lighting', 0.6755456328392029),
 ('editing', 0.6436839699745178),
 ('shot', 0.5833352208137512),
 ('photography', 0.4870421290397644),
 ('sound', 0.47284603118896484),
 ('color', 0.44523200392723083),
 ('visual', 0.43755078315734863),
 ('eye', 0.42765504121780396),
 ('cinematography', 0.42734140157699585)]

In [133]:
model.wv.most_similar("comedy")

[('humor', 0.665596604347229),
 ('funny', 0.6000930070877075),
 ('laughs', 0.5779086351394653),
 ('jokes', 0.5689934492111206),
 ('humorous', 0.5526374578475952),
 ('drama', 0.5477335453033447),
 ('witty', 0.5106291770935059),
 ('amusing', 0.5090083479881287),
 ('hilarious', 0.5065881013870239),
 ('clever', 0.48834213614463806)]

In [134]:
model.wv.most_similar("music")

[('soundtrack', 0.8204672336578369),
 ('songs', 0.7463189363479614),
 ('score', 0.6896651983261108),
 ('song', 0.6471633911132812),
 ('musical', 0.6029199361801147),
 ('cinematography', 0.5782470703125),
 ('sound', 0.5669753551483154),
 ('editing', 0.5594950914382935),
 ('photography', 0.5474770665168762),
 ('singing', 0.5190368294715881)]

# Accessing the vocabulary

In [135]:
list(model.wv.vocab.keys())

['movie',
 'gets',
 'respect',
 'sure',
 'lot',
 'memorable',
 'gem',
 'imagine',
 'joe',
 'actually',
 'funny',
 'scene',
 'character',
 'watch',
 'alan',
 'police',
 'bizarre',
 'horror',
 'filled',
 'famous',
 'later',
 'road',
 'pretty',
 'somewhat',
 'smile',
 'hell',
 'scenes',
 'mood',
 'music',
 'perfect',
 'charming',
 'moves',
 'creepy',
 'things',
 'start',
 'including',
 'couple',
 'great',
 'fun',
 'detective',
 'nearly',
 'baby',
 'based',
 'brought',
 'director',
 'michael',
 'ending',
 'solid',
 'film',
 'wonderful',
 'favorite',
 'thing',
 'way',
 'playing',
 'loved',
 'strange',
 'feeling',
 'sit',
 'theater',
 'parents',
 'kids',
 'felt',
 'like',
 'instead',
 'given',
 'br',
 'book',
 'starts',
 'story',
 'moving',
 'new',
 'town',
 'kid',
 'billy',
 'popular',
 'making',
 'friends',
 'easy',
 'especially',
 'face',
 'death',
 'kill',
 'youre',
 'evil',
 'plays',
 'cool',
 'time',
 'hot',
 'finally',
 'live',
 'dream',
 'living',
 'youve',
 'met',
 'youll',
 'know',

# Accessing word vectors for a given word

In [136]:
model.wv["camera"]

array([-1.6724387 , -3.1794004 ,  1.9912795 ,  2.9107053 , -0.25935468,
       -2.679425  ,  4.9882936 , -1.0490887 ,  0.04670044,  5.8611665 ,
        2.1132116 , -3.5480266 ,  1.5194576 ,  4.1604304 , -0.49718046,
        0.82010233, -4.2324853 ,  2.5300727 ,  2.2998767 ,  6.1309385 ,
        1.9496654 , -1.5353881 ,  2.607237  , -1.1487209 , -1.0636555 ,
       -2.9479744 ,  6.009049  , -0.4433244 ,  1.4796653 ,  1.6386563 ,
        1.2233264 ,  0.7568268 , -0.40140912,  5.588269  ,  2.1424122 ,
       -2.301741  ,  1.9602966 , -1.9266824 ,  1.8389945 , -1.2910291 ,
        0.7442943 ,  1.3679914 , -3.4835954 ,  0.9373433 , -0.76555806,
       -0.7258156 ,  0.9619247 ,  0.0446269 , -2.2774782 , -0.17076594,
       -1.6456116 ,  1.7113351 ,  5.2412415 , -0.5248626 ,  3.7418878 ,
       -1.2940336 ,  1.0834837 ,  4.0712705 ,  2.8571942 ,  0.2902996 ,
        4.47584   ,  5.042846  , -1.1748103 ,  4.3885145 ,  0.87972283,
       -5.7937994 ,  1.7607218 ,  2.073596  ,  2.480191  ,  0.21

# Saving Model

In [140]:
from gensim.test.utils import get_tmpfile
path = get_tmpfile("word2vec.model")

model.save("models/gensim_models/word2Vec.model")

# Loading the saved model 

In [142]:
model2 = Word2Vec.load("models/gensim_models/word2Vec.model")

In [143]:
assert(model2.wv["camera"].all() == model.wv["camera"].all())

# Some Inference

In [177]:
model.wv.most_similar(positive = ["roles", "actor"],
                       negative = ["role"])

[('actors', 0.7641208171844482),
 ('performances', 0.5943905711174011),
 ('cast', 0.5943597555160522),
 ('supporting', 0.5702968835830688),
 ('actress', 0.5628995895385742),
 ('performance', 0.548073410987854),
 ('casting', 0.5466182231903076),
 ('talent', 0.5177333354949951),
 ('talented', 0.5092837810516357),
 ('john', 0.5083895325660706)]

In [178]:
model.wv.most_similar(positive = ["better", "bad"],
                       negative = ["good"])

[('worse', 0.8134002685546875),
 ('terrible', 0.5607643127441406),
 ('mean', 0.45094192028045654),
 ('far', 0.4078099727630615),
 ('ive', 0.3955225348472595),
 ('poor', 0.3855118155479431),
 ('probably', 0.3752049207687378),
 ('possibly', 0.37369170784950256),
 ('cheesy', 0.3717992305755615),
 ('movies', 0.3697521984577179)]

In [176]:
model.wv.most_similar(positive = ["actor", "female"],
                       negative = ["male"])

[('role', 0.7264748811721802),
 ('actress', 0.6643605828285217),
 ('performance', 0.6232424378395081),
 ('roles', 0.6094292402267456),
 ('job', 0.5541602373123169),
 ('actors', 0.5520594120025635),
 ('villain', 0.5445796251296997),
 ('john', 0.5269485712051392),
 ('robert', 0.5259082317352295),
 ('plays', 0.5250316858291626)]